In [1]:
import requests
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re

In [109]:
cards_df = pd.read_csv(".\\data\\main_card_data.csv")
# cards_df['set'] = np.where((cards_df.set == 'Ikoria: Lair of Behemoths'),'Ikoria Lair of Behemoths',cards_df.set)
# cards_df['set'] = np.where((cards_df.set == "Summer Magic / Edgar"),'Summer Magic',cards_df.set)
# cards_df['set'] = np.where((cards_df.set == 'Ravnica: City of Guilds'),'Ravnica City of Guilds',cards_df.set)

sets_up_next = cards_df.set.unique()
# sets_up_next = sets_up_next[np.where((sets_up_next == "Limited Edition Alpha"))]
print(sets_up_next)


['Core Set 2020' 'Core Set 2019' 'Magic Origins' 'Magic 2015' 'Magic 2014'
 'Magic 2013' 'Magic 2012' 'Magic 2011' 'Magic 2010' 'Tenth Edition'
 'Ninth Edition' 'Eighth Edition' 'Seventh Edition'
 'Classic Sixth Edition' 'Fifth Edition' 'Fourth Edition' 'Summer Magic'
 'Revised Edition' 'Unlimited Edition' 'Limited Edition Beta'
 'Limited Edition Alpha' 'Ikoria Lair of Behemoths' 'Theros Beyond Death'
 'Throne of Eldraine' 'War of the Spark' 'Ravnica Allegiance'
 'Guilds of Ravnica' 'Dominaria' 'Rivals of Ixalan' 'Ixalan'
 'Hour of Devastation' 'Amonkhet' 'Aether Revolt' 'Kaladesh'
 'Eldritch Moon' 'Shadows over Innistrad' 'Oath of the Gatewatch'
 'Battle for Zendikar' 'Dragons of Tarkir' 'Fate Reforged'
 'Khans of Tarkir' 'Journey into Nyx' 'Born of the Gods' 'Theros'
 "Dragon's Maze" 'Gatecrash' 'Return to Ravnica' 'Avacyn Restored'
 'Dark Ascension' 'Innistrad' 'New Phyrexia' 'Mirrodin Besieged'
 'Scars of Mirrodin' 'Rise of the Eldrazi' 'Worldwake' 'Zendikar'
 'Alara Reborn' 'Confl

In [101]:
chrome_options = Options()
chrome_options.add_argument("--headless")
browser = webdriver.Chrome('c:/chromedriver.exe', options=chrome_options)
browser.implicitly_wait(30)

all_cards = []

for set_ in sets_up_next:
    print(set_)
    card_names = cards_df[cards_df["set"] == set_].name
    cards = []
    names = []
    for i in card_names:
        name = re.sub("[,'\.`\\\/]", "", i.lower().replace(" ", "-"))
        name = re.sub("á", "a", name)
        name = re.sub("â", "a", name)
        name = re.sub("ú", "u", name)
        name = re.sub("é", "e", name)
        name = re.sub("û", "u", name)
        name = re.sub("ö", "o", name)
        name = re.sub("í", "i", name)
        if (name in cards) | (name in all_cards) | (name == "chaos-orb"):
            continue
        cards.append(name)
        names.append(i)
        all_cards.append(name)
                       
    edhrec_presence = []
    for c in cards:
        print(c)
        browser.get("https://edhrec.com/cards/{}".format(c))
        try:
            nav = browser.find_element_by_xpath("//*[@id='root']/div/div[4]/div[2]/div/div[2]/div[2]/div[1]/div[1]/div[1]/div/div[4]")
            txt = nav.text
            n = int(re.search("\d+", txt).group(0))
            edhrec_presence.append(n)
            print(n)
        except AttributeError:
            print("weewoo")
            nav = browser.find_element_by_xpath("//*[@id='root']/div/div[4]/div[2]/div/div[2]/div[2]/div[1]/div[1]/div[1]/div/div[5]")
            txt = nav.text
            n = int(re.search("\d+", txt).group(0))
            edhrec_presence.append(n)
            print(n)
      
        
    data = {"names": names, "edhrec": edhrec_presence}
    df = pd.DataFrame(data)
    df.to_csv(".\data\cards\edhrec\{}_edh_info.csv".format(set_), index = False)
                       
browser.quit()

Limited Edition Alpha
animate-wall
113
armageddon
6181
balance
weewoo
4
benalish-hero
42
black-ward
82
blaze-of-glory
18
blessing
88
blue-ward
83
castle
256
circle-of-protection:-blue
141
circle-of-protection:-green
171
circle-of-protection:-red
186
circle-of-protection:-white
333
consecrate-land
69
conversion
23
crusade
866
death-ward
195
disenchant
7542
farmstead
2
green-ward
77
guardian-angel
7
healing-salve
52
holy-armor
18
holy-strength
97
island-sanctuary
1486
karma
230
lance
11
mesa-pegasus
53
northern-paladin
150
pearled-unicorn
14
personal-incarnation
13
purelace
5
red-ward
80
resurrection
759
reverse-damage
121
righteousness
265
samite-healer
23
savannah-lions
436
serra-angel
910
swords-to-plowshares
94323
veteran-bodyguard
38
wall-of-swords
230
white-knight
578
white-ward
40
wrath-of-god
26504
air-elemental
132
ancestral-recall
weewoo
5
animate-artifact
21
blue-elemental-blast
644
braingeyser
1005
clone
4476
control-magic
4356
copy-artifact
4325
counterspell
88169
creature-b

In [110]:
cols = ["names", "edhrec"]
all_edhrec_df = pd.DataFrame(columns = cols)
for set_ in cards_df.set.unique():
    edhrec_set_df = pd.read_csv("./data/cards/edhrec/{}".format(set_ + "_edh_info.csv")) 
    all_edhrec_df = pd.concat([all_edhrec_df, edhrec_set_df])
len(all_edhrec_df.names.unique())

18309

In [111]:
all_edhrec_df.to_csv("./data/edhrec_data.csv", index = False)

In [68]:
chrome_options = Options()
chrome_options.add_argument("--headless")
browser = webdriver.Chrome('c:/chromedriver.exe', options=chrome_options)
browser.implicitly_wait(20)

browser.get("https://edhrec.com/cards/blisterspit-gremlin")
try:
    nav = browser.find_element_by_xpath("//*[@id='root']/div/div[4]/div[2]/div/div[2]/div[2]/div[1]/div[1]/div[1]/div/div[4]")
    txt = nav.text
    n = int(re.search("\d+", txt).group(0))
    print(n)
except AttributeError:
    nav = browser.find_element_by_xpath("//*[@id='root']/div/div[4]/div[2]/div/div[2]/div[2]/div[1]/div[1]/div[1]/div/div[5]")
    txt = nav.text
    n = int(re.search("\d+", txt).group(0))
    print(n)
# edhrec_presence.append(n)

browser.quit()

69
